In [81]:
# from google.colab import drive
# drive.mount("/gdrive", force_remount=True)

In [82]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
from keras.datasets import mnist

class MNIST(nn.Module):

  def __init__(self, config):
    ######### 코드 작성 #########
    super(MNIST, self).__init__()
    #############################

    # 입력층 노드 수
    self.inode = config["input_node"]
    # 은닉층 데이터 크기
    self.hnode = config["hidden_node"]
    # 출력층 노드 수: 분류해야 하는 레이블 수
    self.onode = config["output_node"]

    # 활성화 함수로 Sigmoid 사용
    self.activation = nn.Sigmoid()

    # 신경망 설계
    self.linear1 = nn.Linear(self.inode, self.hnode, bias=True)
    # self.linear2 = nn.Linear(self.hnode, self.hnode, bias=True)
    # self.linear3 = nn.Linear(self.hnode, self.hnode, bias=True)
    # self.linear4 = nn.Linear(self.hnode, self.hnode, bias=True)
    # self.linear5 = nn.Linear(self.hnode, self.hnode, bias=True)
    self.classifier = nn.Linear(self.hnode, self.onode, bias=True)

  def forward(self, x):

    x = self.linear1(x)
    x = self.activation(x)

    # x = self.linear2(x)
    # x = self.activation(x)

    # x = self.linear3(x)
    # x = self.activation(x)

    # x = self.linear4(x)
    # x = self.activation(x)

    # x = self.linear5(x)
    # x = self.activation(x)

    x = self.classifier(x)
    x = self.activation(x)

    return x

In [83]:

# 데이터 읽기 함수
def load_dataset():
  ######### 코드 작성 #########
  #mnist.load_data(path="mnist.npz")
  (train_X, train_y), (test_X, test_y) = mnist.load_data()
  #############################

  train_X = train_X.reshape(-1, 28*28)
  print(train_X.shape)
  test_X  = test_X.reshape(-1, 28*28)

  train_X = torch.tensor(train_X, dtype=torch.float)
  train_y = torch.tensor(train_y, dtype=torch.long)
  test_X = torch.tensor(test_X, dtype=torch.float)
  test_y = torch.tensor(test_y, dtype=torch.long)

  return (train_X, train_y), (test_X, test_y)

(train_x, train_y), (test_x, test_y) = load_dataset()
print(train_x.shape)
print(train_y.shape)

print(test_x.shape)
print(test_y.shape)


(60000, 784)
torch.Size([60000, 784])
torch.Size([60000])
torch.Size([10000, 784])
torch.Size([10000])


In [84]:
# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

# 평가 수행 함수
def do_test(model, test_dataloader):

  # 평가 모드 셋팅
  model.eval()

  # Batch 별로 예측값과 정답을 저장할 리스트 초기화
  predicts, golds = [], []

  with torch.no_grad():

    for step, batch in enumerate(test_dataloader):

      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      input_features, labels = batch
      hypothesis = model(input_features)

      # ont-hot 표현으로 변경
      logits = torch.argmax(hypothesis,-1)

      x = tensor2list(logits)
      y = tensor2list(labels)

      # 예측값과 정답을 리스트에 추가
      predicts.extend(x)
      golds.extend(y)

    print("PRED=",predicts)
    print("GOLD=",golds)
    print("Accuracy= {0:f}\n".format(accuracy_score(golds, predicts)))

# 모델 평가 함수
def test(config):

  ######### 코드 작성 #########
  model = MNIST(config)
  model = model.cuda()
  #############################

  # 저장된 모델 가중치 로드
  model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])))

  ######### 코드 작성 #########
  _, (features, labels) = load_dataset()
  #############################
  
  test_features = TensorDataset(features, labels)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])

  do_test(model, test_dataloader)

In [85]:
# 모델 학습 함수
def train(config):

  ######### 코드 작성 #########
  (input_features, labels), _ = load_dataset()
  #############################
  
  # TensorDataset/DataLoader를 통해 배치(batch) 단위로 데이터를 나누고 셔플(shuffle)
  train_features = TensorDataset(input_features, labels)
  train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])


  ######### 코드 작성 #########
  model = MNIST(config)
  model = model.cuda()
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learn_rate'])
  #############################
  
  for epoch in range(config["epoch"]+1):

      ######### 코드 작성 #########
    model.train()
    costs = []

      #############################

    for (step, batch) in enumerate(train_dataloader):

        ######### 코드 작성 #########
      x = batch[0].cuda()
      y = batch[1].cuda()

      optimizer.zero_grad()
      pred_y = model(x)
      cost = criterion(pred_y, y) 
      cost.backward()
      optimizer.step()
        #############################

      # 현재 batch의 스텝 별 loss 저장
      costs.append(cost.data.item())

    # 에폭마다 평균 비용 출력하고 모델을 저장
    print("Average Loss= {0:f}".format(np.mean(costs)))
    torch.save(model.state_dict(), os.path.join(config["output_dir"], "epoch_{0:d}.pt".format(epoch)))
    do_test(model, train_dataloader)

In [86]:
if(__name__=="__main__"):

    root_dir = ""
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "model_name":"epoch_{0:d}.pt".format(10),
              "output_dir":output_dir,
              "input_node":784,
              "hidden_node":512,
              "output_node":10,
              "learn_rate":0.001,
              "batch_size":32,
              "epoch":10,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

(60000, 784)
Average Loss= 1.618793
PRED= [5, 1, 3, 4, 0, 6, 1, 7, 9, 8, 4, 2, 8, 0, 8, 7, 1, 8, 9, 3, 0, 2, 5, 2, 1, 0, 9, 4, 4, 5, 5, 5, 7, 5, 8, 8, 1, 4, 1, 7, 7, 5, 7, 1, 3, 0, 3, 5, 6, 2, 6, 5, 2, 5, 9, 8, 4, 4, 9, 1, 6, 6, 7, 9, 9, 5, 1, 0, 6, 3, 7, 4, 8, 6, 1, 4, 3, 9, 0, 2, 0, 0, 7, 5, 9, 0, 3, 2, 6, 0, 4, 9, 4, 8, 1, 1, 2, 1, 7, 4, 6, 1, 0, 4, 0, 6, 4, 0, 6, 0, 2, 9, 9, 6, 9, 8, 6, 8, 7, 6, 1, 6, 9, 2, 1, 2, 6, 6, 1, 1, 9, 7, 2, 5, 5, 2, 1, 1, 0, 6, 8, 8, 8, 0, 2, 5, 3, 6, 7, 6, 8, 5, 1, 9, 8, 6, 0, 2, 2, 2, 2, 2, 1, 1, 2, 0, 3, 3, 1, 3, 2, 2, 4, 1, 0, 9, 4, 9, 2, 7, 7, 3, 8, 8, 5, 0, 9, 7, 4, 9, 0, 4, 4, 2, 6, 4, 6, 4, 6, 6, 4, 8, 3, 6, 0, 7, 7, 9, 9, 7, 7, 2, 7, 4, 2, 0, 2, 1, 0, 0, 6, 2, 1, 7, 1, 8, 3, 3, 7, 2, 0, 2, 8, 2, 8, 7, 1, 3, 1, 5, 4, 9, 3, 9, 0, 2, 6, 4, 4, 6, 4, 3, 2, 4, 7, 3, 9, 0, 9, 5, 9, 6, 8, 4, 2, 1, 8, 8, 9, 8, 5, 7, 1, 8, 5, 9, 5, 5, 4, 2, 5, 1, 5, 1, 4, 7, 3, 0, 8, 0, 0, 5, 2, 6, 2, 1, 4, 1, 0, 4, 4, 3, 1, 5, 2, 6, 8, 9, 4, 7, 6, 4, 7, 5, 1, 6, 5, 3, 7, 

In [87]:
root_dir = ""
output_dir = os.path.join(root_dir, "output")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

config = {"mode": "test",
            "model_name":"epoch_{0:d}.pt".format(10),
            "output_dir":output_dir,
            "input_node":784,
            "hidden_node":512,
            "output_node":10,
            "learn_rate":0.001,
            "batch_size":32,
            "epoch":10,
            }

if(config["mode"] == "train"):
    train(config)
else:
    test(config)

(60000, 784)
PRED= [6, 2, 0, 6, 5, 1, 8, 1, 5, 1, 5, 0, 1, 2, 1, 2, 8, 0, 2, 3, 4, 2, 5, 2, 6, 9, 7, 3, 6, 8, 5, 4, 7, 2, 8, 3, 2, 1, 3, 4, 1, 2, 9, 1, 0, 3, 4, 7, 6, 3, 1, 6, 4, 5, 7, 1, 1, 5, 2, 8, 1, 9, 3, 7, 8, 4, 7, 7, 0, 6, 2, 8, 9, 8, 8, 5, 4, 1, 8, 8, 3, 7, 6, 1, 2, 3, 1, 0, 3, 9, 9, 0, 9, 7, 9, 1, 7, 0, 0, 8, 6, 1, 4, 2, 0, 6, 9, 6, 6, 1, 7, 6, 4, 8, 6, 4, 8, 2, 3, 9, 4, 2, 3, 9, 9, 8, 4, 8, 3, 7, 0, 2, 6, 2, 4, 2, 6, 8, 6, 0, 1, 6, 1, 3, 2, 2, 2, 7, 1, 0, 3, 9, 5, 9, 7, 3, 3, 4, 3, 9, 1, 6, 8, 1, 8, 4, 1, 1, 3, 2, 0, 8, 0, 6, 1, 9, 1, 5, 0, 5, 0, 1, 2, 1, 5, 5, 6, 1, 8, 1, 9, 8, 3, 6, 5, 7, 3, 6, 4, 0, 5, 3, 7, 1, 3, 5, 0, 7, 9, 8, 6, 1, 0, 1, 0, 3, 9, 4, 3, 7, 0, 8, 4, 0, 6, 3, 8, 4, 8, 5, 4, 8, 3, 4, 4, 6, 4, 4, 4, 2, 3, 8, 0, 2, 5, 8, 5, 3, 4, 2, 0, 4, 1, 4, 8, 7, 9, 3, 0, 5, 2, 1, 4, 9, 9, 0, 6, 3, 8, 2, 8, 1, 1, 7, 3, 5, 6, 1, 1, 4, 0, 1, 9, 6, 9, 5, 4, 6, 4, 1, 9, 8, 6, 1, 9, 8, 4, 2, 4, 3, 9, 1, 3, 4, 4, 3, 0, 9, 2, 5, 2, 5, 3, 9, 7, 2, 6, 4, 6, 1, 4, 5, 2, 5, 6, 7, 5,